Clinical Only

No exploratory data analysis was performed as entries with missing data were removed, the data was already normalized and removing outliers would not be ideal as the normal varience in the population is uknown for all 62000 transcripts. 

In [ ]:
# Importing the libraries
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')

from sklearn.utils import resample
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold,RandomizedSearchCV
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import shap
from collections import defaultdict


In [ ]:
#Added relative path, place path where data is in this line.
data = pd.read_csv("C:/Users/migue/OneDrive/Desktop/ML Project/Counts/Individual Files/Extracted/Invidual/ML_project_Clinical only.csv")
print(data)
#Need to work on this.
x_frame = data.iloc[: , :-1]
#x_frame = x_frame.to_numpy()

y_frame = data.iloc[:,-1:]


print(x_frame)
print(y_frame)

print(x_frame.shape)

print(y_frame.shape)

In [ ]:
#Set varibles for data scaling and  stratified Kfold for the hw examples.
# Will need to do boostrapping not stratified kfold
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
le = preprocessing.LabelEncoder()
oe = OrdinalEncoder()
x_frame = oe.fit_transform(x_frame)
print(y_frame)
y_frame = le.fit_transform(y_frame)
print(y_frame)
#(categories = ['Age at Index','Age of Diagnosis in Days','Days from Birth','Days to Last Follow UP','Gender','Sample Type','Treament Or Therapy','Treatment Type','Vital Status'])
cv_outer = StratifiedKFold(n_splits=6, shuffle=True, random_state=1)
cv_inner = StratifiedKFold(n_splits=6, shuffle=True, random_state=2)

In [ ]:
#Uses CV inner to determine best parameters for xgb using the RandomizedSearchCV parameter optimization. 
model_xgb = xgb.XGBClassifier(random_state=1,objective='multi:softmax',eval_metric='logloss',use_label_encoder=False,num_class =8)
xgb_pipeline = Pipeline([
    ('pca', PCA()), 
    ('model_xgb', model_xgb,)
])

xgb_param= {
    'pca__n_components': [0.60,0.70,0.80,0.90],
    'model_xgb__max_depth': [2, 3, 5,7,9],
    'model_xgb__n_estimators': [10, 100, 500],
    'model_xgb__average':['none','micro','macro','weighted']}
xgb_random = RandomizedSearchCV(xgb_pipeline, xgb_param, cv=cv_inner, scoring='accuracy')
print(xgb_random)
print()

In [ ]:
#Code below is an example for the hw assignment for how to perform nested k fold on the stratified k fold splits previously.
#Sets variables for the following for loop in the code below. 
import warnings 
#Silences warnings. 
warnings.filterwarnings('ignore')

rf_outer_results = list()
rf_best_f1_score=float('-inf')
rf_best_parameters={}

xgb_outer_results = list()
xgb_best_f1_score= float('-inf')
xgb_best_parameters={}

#for loop that runs 1 times that uses the optimized parameters from above to train the models using a new outer split of the data. 
iter_num=1
for train_ix, test_ix in cv_outer.split(x_frame,y_frame):
    print('Iteration',iter_num)
    iter_num +=1
    # split data
    X_train, X_test, y_train, y_test = train_test_split(x_frame, y_frame, random_state = 1)
    print(type(X_test))
    #X_train, X_test = x_frame[train_ix, :], x_frame[test_ix, :]
    #y_train, y_test = y_frame[train_ix], y_frame[test_ix]

    #XGBoost
    result = xgb_random.fit(X_train, y_train)
    best_model = result.best_estimator_
    yhat = best_model.predict(X_test)
    f1score=f1_score(y_test,yhat,average ='weighted')
    if f1score >= xgb_best_f1_score:       
        xgb_best_f1_score=f1score
        xgb_best_parameters=result.best_params_
    xgb_outer_results.append(f1score)
    print('XGB inner test: est=%.3f, cfg=%s' % (result.best_score_, result.best_params_))
    print()

# summarize the estimated performance of the model

xgb_mean=np.mean(xgb_outer_results)

print('XGB outer test: f1-score mean: %.3f (std: %.3f)' % (xgb_mean, np.std(xgb_outer_results)))
    

In [ ]:
#Find the highest mean between the trained models, it then returns the best performing model based on that. 
defaulttParm=model_xgb.get_xgb_params()
pca=PCA(xgb_best_parameters['pca__n_components'])
for k in xgb_best_parameters.keys():
    if 'model_xgb' in k:
        parm=k.split('__')[1]
        defaulttParm[parm]=xgb_best_parameters[k]
defaulttParm['use_label_encoder']=False
myFinalModel=xgb.XGBClassifier(**defaulttParm)

In [ ]:
#PCA
x_pca=pca.fit_transform(x_frame)

In [ ]:
#prints the cross validation scores.
print(cross_val_score(myFinalModel,x_pca, y_frame,cv=9))

In [ ]:
print(cross_val_score(myFinalModel,x_frame, y_frame,cv=9))

In [ ]:
# Determines the f-score of the model using a new tenth split. 
X_train, X_test, y_train, y_test = train_test_split(x_frame, y_frame, test_size=0.1, random_state=1)
myFinalModel.fit(X_train,y_train)
yhat=myFinalModel.predict(X_test)
f1score=f1_score(yhat,y_test, average ='weighted')
print(f1score)

In [ ]:
#Perform Shap analysis on the predictiors of the data for the class = 0, as it the model balances itself to the final prediction.
#Each predictor either increase (red) or decreases(blue) the prediction score. 
shap.initjs()
explainer = shap.KernelExplainer(myFinalModel.predict,X_train)
print(type(explainer))
shap_values = explainer.shap_values(X_test[0:,])

shap.force_plot(explainer.expected_value, shap_values[0], X_test[0,:])


In [ ]:
#Perform Shap analysis on the predictiors of the data for the class = 0, as it the model balances itself to the final prediction.
#Each predictor either increase (red) or decreases(blue) the prediction score. 
shap.initjs()
explainer = shap.KernelExplainer(myFinalModel.predict,X_train)
shap_values = explainer.shap_values(X_test[0:,])
shap.force_plot(explainer.expected_value, shap_values[1], X_test[0,:])

In [ ]:
#This code produces a beeswarm plot showing the impact of the top 10 features on the model when its value is 0.
#creates the explainer object
explainer = shap.TreeExplainer(myFinalModel)
#produces values from the test set. 
shap_values = explainer.shap_values(X_test)
#creates the shap object for the plot
shap_object = explainer(X_test)
#produces the plot
shap.summary_plot(shap_values[0],X_test)


In [ ]:
#This code produces a beeswarm plot showing the impact of the top 10 features on the model when its value is 1.
#creates the explainer object
explainer = shap.TreeExplainer(myFinalModel)
#produces values from the test set. 
shap_values = explainer.shap_values(X_test)
#creates the shap object for the plot
shap_object = explainer(X_test)
#produces the plot
shap.summary_plot(shap_values[1],X_test)

In [ ]:
#Uses the learning curve function from the yellowbrick library to identify if the model could benefit from more training samples. 
from yellowbrick.model_selection import learning_curve
#prints the learning curve plot. 
print(learning_curve(myFinalModel,X_test, y_test,cv=10,))

In [ ]:
#import libraries 
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from matplotlib import pyplot
# create dataset with n_samples. 
X, y = make_classification(n_samples=100000, random_state=1)
# split into train test sets 80 20 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# define lists to collect accuracy scores. 
train_scores, test_scores = list(), list()
# define the tree depths to evaluate
values = [i for i in range(1, 21)]
#for loop that takes split above and plots train vs test to determine fit.
for i in values:
    myFinalModel.fit(X_train, y_train)
    # evaluate on the train dataset
    train_yhat = myFinalModel.predict(X_train)
    #determines accuracy score of training data set model.
    train_acc = accuracy_score(y_train, train_yhat)
    #appends train accuracy score to list. 
    train_scores.append(train_acc)
    # evaluate on the test dataset
    test_yhat = myFinalModel.predict(X_test)
    #determines accuracy score of test data set model 
    test_acc = accuracy_score(y_test, test_yhat)
    #appends the test accuracy score to list. 
    test_scores.append(test_acc)
    # summarize progress
    print('>%d, train: %.3f, test: %.3f' % (i, train_acc, test_acc))
# plot of train and test scores 
pyplot.plot(values, train_scores, '-o', label='Train')
pyplot.plot(values, test_scores, '-o', label='Test')
pyplot.legend()
pyplot.show()

Clinical plus transport transcripts 

In [ ]:
# Importing the libraries
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')

from sklearn.utils import resample
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold,RandomizedSearchCV
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import shap
from collections import defaultdict

In [ ]:
#Added relative path, place path where data is in this line.
data = pd.read_csv("C:/Users/migue/OneDrive/Desktop/ML Project/Counts/Individual Files/Extracted/Invidual/transposed_transport_genes.csv")
print(data)
#Need to work on this.
x_frame = data.iloc[: , :-1]
#x_frame = x_frame.to_numpy()

y_frame = data.iloc[:,-1:]


print(x_frame)
print(y_frame)

print(x_frame.shape)

print(y_frame.shape)

In [ ]:
#Set varibles for data scaling and  stratified Kfold for the hw examples.
# Will need to do boostrapping not stratified kfold
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
le = preprocessing.LabelEncoder()
oe = OrdinalEncoder()
x_frame = oe.fit_transform(x_frame)
print(y_frame)
y_frame = le.fit_transform(y_frame)
print(y_frame)
#(categories = ['Age at Index','Age of Diagnosis in Days','Days from Birth','Days to Last Follow UP','Gender','Sample Type','Treament Or Therapy','Treatment Type','Vital Status'])
cv_outer = StratifiedKFold(n_splits=2, shuffle=True, random_state=1)
cv_inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=2)


In [ ]:
#Uses CV inner to determine best parameters for xgb using the RandomizedSearchCV parameter optimization. 
model_xgb = xgb.XGBClassifier(random_state=1,objective='multi:softmax',eval_metric='mlogloss',use_label_encoder=False,num_class =8)
xgb_pipeline = Pipeline([
    ('pca', PCA()), 
    ('model_xgb', model_xgb,)
])

xgb_param= {
    'pca__n_components': [0.60,0.70,0.80,0.90],
    'model_xgb__max_depth': [2, 3, 5,7,9],
    'model_xgb__n_estimators': [10, 100, 500],
    'model_xgb__average':['none','micro','macro','weighted']}
xgb_random = RandomizedSearchCV(xgb_pipeline, xgb_param, cv=cv_inner, scoring='accuracy')
print(xgb_random)
print()

In [ ]:
#Code below is an example for the hw assignment for how to perform nested k fold on the stratified k fold splits previously.
#Sets variables for the following for loop in the code below. 
import warnings 
#Silences warnings. 
warnings.filterwarnings('ignore')

rf_outer_results = list()
rf_best_f1_score=float('-inf')
rf_best_parameters={}

xgb_outer_results = list()
xgb_best_f1_score= float('-inf')
xgb_best_parameters={}

#for loop that runs 1 times that uses the optimized parameters from above to train the models using a new outer split of the data. 
iter_num=1
for train_ix, test_ix in cv_outer.split(x_frame,y_frame):
    print('Iteration',iter_num)
    iter_num +=1
    # split data
    X_train, X_test, y_train, y_test = train_test_split(x_frame, y_frame, train_size =0.8,test_size =0.2, random_state = 1)
    #X_train, X_test = x_frame[train_ix, :], x_frame[test_ix, :]
    #y_train, y_test = y_frame[train_ix], y_frame[test_ix]

    #XGBoost
    result = xgb_random.fit(X_train, y_train)
    best_model = result.best_estimator_

    yhat = best_model.predict(X_test)
    f1score=f1_score(y_test,yhat,average ='weighted')
    if f1score >= xgb_best_f1_score:       
        xgb_best_f1_score=f1score
        xgb_best_parameters=result.best_params_
    xgb_outer_results.append(f1score)
    print('XGB inner test: est=%.3f, cfg=%s' % (result.best_score_, result.best_params_))
    print()

# summarize the estimated performance of the model

xgb_mean=np.mean(xgb_outer_results)

print('XGB outer test: f1-score mean: %.3f (std: %.3f)' % (xgb_mean, np.std(xgb_outer_results)))
    

In [ ]:
#Find the highest mean between the trained models, it then returns the best performing model based on that. 
defaulttParm=model_xgb.get_xgb_params()
pca=PCA(xgb_best_parameters['pca__n_components'])
for k in xgb_best_parameters.keys():
    if 'model_xgb' in k:
        parm=k.split('__')[1]
        defaulttParm[parm]=xgb_best_parameters[k]
defaulttParm['use_label_encoder']=False
myFinalModel=xgb.XGBClassifier(**defaulttParm)

In [ ]:
#PCA
x_pca=pca.fit_transform(x_frame)

In [ ]:
print(cross_val_score(myFinalModel,x_frame, y_frame,cv=9))

In [ ]:
# Determines the f-score of the model using a new tenth split. 
X_train, X_test, y_train, y_test = train_test_split(x_frame, y_frame, test_size=0.1, random_state=1)
myFinalModel.fit(X_train,y_train)
yhat=myFinalModel.predict(X_test[0:,])
print(yhat)
f1score=f1_score(yhat,y_test, average ='weighted')
print(f1score)

In [ ]:
#Perform Shap analysis on the predictiors of the data for the class = 0, as it the model balances itself to the final prediction.
#Each predictor either increase (red) or decreases(blue) the prediction score. 
shap.initjs()
explainer = shap.KernelExplainer(myFinalModel.predict,X_train)

shap_values = explainer.shap_values(X_test[0:,])

shap.force_plot(explainer.expected_value, shap_values[1], X_test[0,:])

In [ ]:
#Perform Shap analysis on the predictiors of the data for the class = 0, as it the model balances itself to the final prediction.
#Each predictor either increase (red) or decreases(blue) the prediction score. 
shap.initjs()
explainer = shap.KernelExplainer(myFinalModel.predict,X_train)
shap_values = explainer.shap_values(X_test[0:,])
shap.force_plot(explainer.expected_value, shap_values[10], X_test[0,:])

In [ ]:
#This code produces a beeswarm plot showing the impact of the top 10 features on the model when its value is 0.
#creates the explainer object
explainer = shap.TreeExplainer(myFinalModel)
#produces values from the test set. 
shap_values = explainer.shap_values(X_test)
#creates the shap object for the plot
shap_object = explainer(X_test)
#produces the plot
shap.summary_plot(shap_values[0],X_test)

In [ ]:
#This code produces a beeswarm plot showing the impact of the top 10 features on the model when its value is 1.
#creates the explainer object
explainer = shap.TreeExplainer(myFinalModel)
#produces values from the test set. 
shap_values = explainer.shap_values(X_test)
#creates the shap object for the plot
shap_object = explainer(X_test)
#produces the plot
shap.summary_plot(shap_values[1],X_test)

In [ ]:
shap_values = explainer.shap_values(X_test)
shap_object = explainer(X_test)

shap.summary_plot(shap_values[1],X_test)

In [ ]:
#Uses the learning curve function from the yellowbrick library to identify if the model could benefit from more training samples. 

from yellowbrick.model_selection import learning_curve

print(learning_curve(myFinalModel,x_frame, y_frame,cv=10,))

In [ ]:
#import libraries 
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from matplotlib import pyplot
# create dataset with n_samples. 
X, y = make_classification(n_samples=100000, random_state=1)
# split into train test sets 80 20 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# define lists to collect accuracy scores. 
train_scores, test_scores = list(), list()
# define the tree depths to evaluate
values = [i for i in range(1, 21)]
#for loop that takes split above and plots train vs test to determine fit.
for i in values:
    myFinalModel.fit(X_train, y_train)
    # evaluate on the train dataset
    train_yhat = myFinalModel.predict(X_train)
    #determines accuracy score of training data set model.
    train_acc = accuracy_score(y_train, train_yhat)
    #appends train accuracy score to list. 
    train_scores.append(train_acc)
    # evaluate on the test dataset
    test_yhat = myFinalModel.predict(X_test)
    #determines accuracy score of test data set model 
    test_acc = accuracy_score(y_test, test_yhat)
    #appends the test accuracy score to list. 
    test_scores.append(test_acc)
    # summarize progress
    print('>%d, train: %.3f, test: %.3f' % (i, train_acc, test_acc))
# plot of train and test scores 
pyplot.plot(values, train_scores, '-o', label='Train')
pyplot.plot(values, test_scores, '-o', label='Test')
pyplot.legend()
pyplot.show()

Clincial plus postive regulation of apoptosis

In [ ]:
# Importing the libraries
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')

from sklearn.utils import resample
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold,RandomizedSearchCV
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import shap
from collections import defaultdict


In [ ]:
#Added relative path, place path where data is in this line.
data = pd.read_csv("C:/Users/migue/OneDrive/Desktop/ML Project/Counts/Individual Files/Extracted/Invidual/Transposed_Postive_Regulation_Apoptosis.csv")
print(data)
#Need to work on this.
x_frame = data.iloc[: , :-1]
#x_frame = x_frame.to_numpy()

y_frame = data.iloc[:,-1:]


print(x_frame)
print(y_frame)

print(x_frame.shape)

print(y_frame.shape)

In [ ]:
#Set varibles for data scaling and  stratified Kfold for the hw examples.
# Will need to do boostrapping not stratified kfold
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
le = preprocessing.LabelEncoder()
oe = OrdinalEncoder()
x_frame = oe.fit_transform(x_frame)
print(y_frame)
y_frame = le.fit_transform(y_frame)
print(y_frame)
#(categories = ['Age at Index','Age of Diagnosis in Days','Days from Birth','Days to Last Follow UP','Gender','Sample Type','Treament Or Therapy','Treatment Type','Vital Status'])
cv_outer = StratifiedKFold(n_splits=6, shuffle=True, random_state=1)
cv_inner = StratifiedKFold(n_splits=6, shuffle=True, random_state=2)

In [ ]:
#Uses CV inner to determine best parameters for xgb using the RandomizedSearchCV parameter optimization. 
model_xgb = xgb.XGBClassifier(random_state=1,objective='multi:softmax',eval_metric='logloss',use_label_encoder=False,num_class =8)
xgb_pipeline = Pipeline([
    ('pca', PCA()), 
    ('model_xgb', model_xgb,)
])

xgb_param= {
    'pca__n_components': [0.60,0.70,0.80,0.90],
    'model_xgb__max_depth': [2, 3, 5,7,9],
    'model_xgb__n_estimators': [10, 100, 500],
    'model_xgb__average':['none','micro','macro','weighted']}
xgb_random = RandomizedSearchCV(xgb_pipeline, xgb_param, cv=cv_inner, scoring='accuracy')
print(xgb_random)
print()

In [ ]:
#Code below is an example for the hw assignment for how to perform nested k fold on the stratified k fold splits previously.
#Sets variables for the following for loop in the code below. 
import warnings 
#Silences warnings. 
warnings.filterwarnings('ignore')

rf_outer_results = list()
rf_best_f1_score=float('-inf')
rf_best_parameters={}

xgb_outer_results = list()
xgb_best_f1_score= float('-inf')
xgb_best_parameters={}

#for loop that runs 1 times that uses the optimized parameters from above to train the models using a new outer split of the data. 
iter_num=1
for train_ix, test_ix in cv_outer.split(x_frame,y_frame):
    print('Iteration',iter_num)
    iter_num +=1
    # split data
    X_train, X_test, y_train, y_test = train_test_split(x_frame[train_ix, :], y_frame[train_ix], train_size =0.8,test_size =0.2, random_state = 1)
    #X_train, X_test = x_frame[train_ix, :], x_frame[test_ix, :]
    #y_train, y_test = y_frame[train_ix], y_frame[test_ix]

    #XGBoost
    result = xgb_random.fit(X_train, y_train)
    best_model = result.best_estimator_
    yhat = best_model.predict(X_test)
    f1score=f1_score(y_test,yhat, average = 'weighted')
    if f1score >= xgb_best_f1_score:       
        xgb_best_f1_score=f1score
        xgb_best_parameters=result.best_params_
    xgb_outer_results.append(f1score)
    print('XGB inner test: est=%.3f, cfg=%s' % (result.best_score_, result.best_params_))
    print()

# summarize the estimated performance of the model

xgb_mean=np.mean(xgb_outer_results)

print('XGB outer test: f1-score mean: %.3f (std: %.3f)' % (xgb_mean, np.std(xgb_outer_results)))

In [ ]:
#Find the highest mean between the trained models, it then returns the best performing model based on that. 
defaulttParm=model_xgb.get_xgb_params()
pca=PCA(xgb_best_parameters['pca__n_components'])
for k in xgb_best_parameters.keys():
    if 'model_xgb' in k:
        parm=k.split('__')[1]
        defaulttParm[parm]=xgb_best_parameters[k]
defaulttParm['use_label_encoder']=True
myFinalModel=xgb.XGBClassifier(**defaulttParm)

In [ ]:
#PCA
x_pca=pca.fit_transform(x_frame)

In [ ]:
print(cross_val_score(myFinalModel,x_pca, y_frame,cv=9))

In [ ]:
print(cross_val_score(myFinalModel,x_frame, y_frame,cv=9))

In [ ]:
# Determines the f-score of the model using a new tenth split. 
X_train, X_test, y_train, y_test = train_test_split(x_frame, y_frame, test_size=0.1, random_state=1)
myFinalModel.fit(X_train,y_train)
yhat=myFinalModel.predict(X_test)
f1score=f1_score(yhat,y_test, average ='weighted')
print(f1score)

In [ ]:
#Perform Shap analysis on the predictiors of the data for the class = 0, as it the model balances itself to the final prediction.
#Each predictor either increase (red) or decreases(blue) the prediction score. 
shap.initjs()
explainer = shap.KernelExplainer(myFinalModel.predict,X_train)

shap_values = explainer.shap_values(X_test[0:,])

shap.force_plot(explainer.expected_value, shap_values[0])

In [ ]:
#Perform Shap analysis on the predictiors of the data for the class = 0, as it the model balances itself to the final prediction.
#Each predictor either increase (red) or decreases(blue) the prediction score. 
shap.initjs()
explainer = shap.KernelExplainer(myFinalModel.predict,X_train)
shap_values = explainer.shap_values(X_test[0:,])
shap.force_plot(explainer.expected_value, shap_values[2], X_test[0,:])

In [ ]:
#This code produces a beeswarm plot showing the impact of the top 10 features on the model when its value is 0.
#creates the explainer object
explainer = shap.TreeExplainer(myFinalModel)
#produces values from the test set. 
shap_values = explainer.shap_values(X_test)
#creates the shap object for the plot
shap_object = explainer(X_test)
#produces the plot
shap.summary_plot(shap_values[0],X_test)

In [ ]:
#This code produces a beeswarm plot showing the impact of the top 10 features on the model when its value is 1.
#creates the explainer object
explainer = shap.TreeExplainer(myFinalModel)
#produces values from the test set. 
shap_values = explainer.shap_values(X_test)
#creates the shap object for the plot
shap_object = explainer(X_test)
#produces the plot
shap.summary_plot(shap_values[1],X_test)

In [ ]:
#Uses the learning curve function from the yellowbrick library to identify if the model could benefit from more training samples. 

from yellowbrick.model_selection import learning_curve
#prints the learning curve plot.
print(learning_curve(myFinalModel,x_frame, y_frame,cv=10,))

In [ ]:
#import libraries 
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from matplotlib import pyplot
# create dataset with n_samples. 
X, y = make_classification(n_samples=100000, random_state=1)
# split into train test sets 80 20 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# define lists to collect accuracy scores. 
train_scores, test_scores = list(), list()
# define the tree depths to evaluate
values = [i for i in range(1, 21)]
#for loop that takes split above and plots train vs test to determine fit.
for i in values:
    myFinalModel.fit(X_train, y_train)
    # evaluate on the train dataset
    train_yhat = myFinalModel.predict(X_train)
    #determines accuracy score of training data set model.
    train_acc = accuracy_score(y_train, train_yhat)
    #appends train accuracy score to list. 
    train_scores.append(train_acc)
    # evaluate on the test dataset
    test_yhat = myFinalModel.predict(X_test)
    #determines accuracy score of test data set model 
    test_acc = accuracy_score(y_test, test_yhat)
    #appends the test accuracy score to list. 
    test_scores.append(test_acc)
    # summarize progress
    print('>%d, train: %.3f, test: %.3f' % (i, train_acc, test_acc))
# plot of train and test scores 
pyplot.plot(values, train_scores, '-o', label='Train')
pyplot.plot(values, test_scores, '-o', label='Test')
pyplot.legend()
pyplot.show()

Based on the plots above the models are underfitting and would benefit from more training data. 
The clical only model is the only one that would not greatly improve from more training data based on its learning curve. 